In [1]:
# async
# await
import asyncio
import time
import threading as th
import multiprocessing as mp

In [5]:
# Case 1: just normal Python

import time

def sleep():
    print(f'Time: {time.time() - start:.2f}')
    time.sleep(0.5)

def sum(name, numbers):
    total = 0
    for number in numbers:
        print(f'Task {name}: Computing {total}+{number}')
        sleep()
        total += number
    print(f'Task {name}: Sum = {total}\n')

start = time.time()
tasks = [
    sum("A", [1, 2]),
    sum("B", [1, 2, 3]),
]
end = time.time()
print(f'Time: {end-start:.2f} sec')

Task A: Computing 0+1
Time: 0.00
Task A: Computing 1+2
Time: 0.50
Task A: Sum = 3

Task B: Computing 0+1
Time: 1.00
Task B: Computing 1+2
Time: 1.50
Task B: Computing 3+3
Time: 2.01
Task B: Sum = 6

Time: 2.51 sec


In [6]:
# Case 2: async/await done wrong

import asyncio
import time

async def sleep():
    print(f'Time: {time.time() - start:.2f}')
    time.sleep(0.5)

async def sum(name, numbers):
    total = 0
    for number in numbers:
        print(f'Task {name}: Computing {total}+{number}')
        await sleep()
        total += number
    print(f'Task {name}: Sum = {total}\n')

start = time.time()

loop = asyncio.get_event_loop()
tasks = [
    loop.create_task(sum("A", [1, 2])),
    loop.create_task(sum("B", [1, 2, 3])),
]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

end = time.time()
print(f'Time: {end-start:.2f} sec')

RuntimeError: This event loop is already running

Task A: Computing 0+1
Time: 1.47
Task A: Computing 1+2
Time: 1.97
Task A: Sum = 3

Task B: Computing 0+1
Time: 2.47
Task B: Computing 1+2
Time: 2.97
Task B: Computing 3+3
Time: 3.48
Task B: Sum = 6



In [7]:
# Case 3: async/await done right

import asyncio
import time

async def sleep():
    print(f'Time: {time.time() - start:.2f}')
    await asyncio.sleep(1)

async def sum(name, numbers):
    total = 0
    for number in numbers:
        print(f'Task {name}: Computing {total}+{number}')
        await sleep()
        total += number
    print(f'Task {name}: Sum = {total}\n')

start = time.time()

loop = asyncio.get_event_loop()
tasks = [
    loop.create_task(sum("A", [1, 2])),
    loop.create_task(sum("B", [1, 2, 3])),
]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

end = time.time()
print(f'Time: {end-start:.2f} sec')

RuntimeError: This event loop is already running

Task A: Computing 0+1
Time: 0.08
Task B: Computing 0+1
Time: 0.08
Task A: Computing 1+2
Time: 1.09
Task B: Computing 1+2
Time: 1.09
Task A: Sum = 3

Task B: Computing 3+3
Time: 2.10
Task B: Sum = 6



In [10]:
# Generally, “async” defines a coroutine, and “await” suspends or yields execution to another coroutine from within a coroutine.

# Async/await was introduced in Python version 3.5.

# Async refers to a suite of expressions for working with coroutines.
# This includes:
#     “async def” for defining a coroutine.
#     “async for” for traversing an asynchronous iterable.
#     “async with” for using an asynchronous context manager.
# The “async def” expression defines a coroutine, but the “async for” and “async with” expressions can only be used within a coroutine,
# e.g. within an “async def” expression.

# Note that both async for and async with can only be used inside a coroutine function declared with async def.

# A coroutine defined with the “async def” expression is referred to as a “coroutine function“

# coroutine function: A function which returns a coroutine object.
# A coroutine function may be defined with the async def statement,
# and may contain await, async for, and async with keywords.

# The “async for” expression is used to traverse an asynchronous iterator.
# It is an asynchronous for-loop statement.
# An asynchronous iterator is an iterator that yields awaitables.

# traverse an asynchronous iterator
# async_iterator = """ a function that returns asynchronously, async_iterator """
# async for item in async_iterator:
# 	print(item)

# Await refers to the “await” expression used with coroutines.
# It can only be used within a coroutine and is used to yield execution to an awaitable.
# An awaitable may be another coroutine or a coroutine wrapped in a Task object for independent execution.
# Put another way, await will cause the caller coroutine to suspend execution at that point and wait for the given awaitable to be done.

In [26]:
async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())

asyncio.run(main())

"""
# Immediately
One
One
One
# After one second
Two
Two
Two
"""

'\n# Immediately\nOne\nOne\nOne\n# After one second\nTwo\nTwo\nTwo\n'

In [27]:
async def g():
    # Pause here and come back to g() when f() is ready
    r = await f()
    return r

In [32]:
async def f(x):
    y = await z(x)  # OK - `await` and `return` allowed in coroutines
    return y

async def g(x):
    yield x  # OK - this is an async generator

async def m(x):
    # yield from gen(x)  # No - SyntaxError
    pass

def m(x):
    # y = await z(x)  # Still no - SyntaxError (no `async def` here)
    return y

In [34]:
import asyncio

# @asyncio.coroutine
def py34_coro():
    """Generator-based coroutine, older syntax"""
    yield from stuff()

async def py35_coro():
    """Native coroutine, modern syntax"""
    await stuff()